<h1>Liveability</h1>

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../scripts/')
from helper_functions import convert_census_to_postcode

import geopandas as gpd 
import folium

Obtaining property data

In [2]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/properties_processed.csv")

# Remove unecessary columns "Unnamed: 0", "Name", "Cost", "Coordinates", "Property_Type" & "Agency"
property_df = property_df.drop(columns = ["Unnamed: 0", "Name", "Cost", "Coordinates", "Property_Type", "Agency"])
property_df.head()

,Bed,Bath,Parking,Postcode
0,1,1,0,3000
1,1,1,0,3000
2,1,1,0,3000
3,1,1,0,3000
4,2,2,1,3000


In [22]:
# Reading in preprocessed distance data
distances_df = pd.read_csv("../data/curated/categorised_distances.csv")

# Removing unecessary columns (all columns apart from numerical/categorical distance measures & postcode)
distances_df = distances_df[["Bed", "Bath", "Parking", "Postcode", "station_duration", "cbd_duration", "postoffice_duration", "park_distance", 
                                "Nearby_Schools"]]
distances_df.head()

,Bed,Bath,Parking,Postcode,station_duration,cbd_duration,postoffice_duration,park_distance,Nearby_Schools
0,1,1,0,3000,1,80.0,1,1,612
1,1,1,0,3000,2,113.5,1,1,630
2,1,1,0,3000,1,70.4,1,1,612
3,1,1,0,3000,2,93.4,1,1,625
4,2,2,1,3000,1,136.8,1,1,608


In [14]:
# Reading in preprocessed schools data
schools_df = pd.read_csv("../data/curated/schools/school_counts.csv")
schools_df.head()

,Unnamed: 0,Postcode,Education_Sector,School_Type,Number_of_Schools
0,0,3000,Independent,Secondary,3
1,1,3000,Independent,Special,1
2,2,3003,Catholic,Secondary,1
3,3,3004,Government,Secondary,1
4,4,3004,Government,Special,1


Calculating the average number of nearby amenities

In [16]:
# Calculating the average distance from the CBD per postcode
avg_cbd_dur = distances_df.groupby(by = "Postcode")["cbd_duration"].mean()
avg_cbd_dur.head()

Postcode
3000    130.582394
3002    302.665909
3003    228.284127
3004    431.089247
3006    311.044615
Name: cbd_duration, dtype: float64

In [17]:
# Calculating the average number of amenities per postcode
avg_station_dur = distances_df.groupby(by = "Postcode")["station_duration"].mean()
avg_station_dur.head()

Postcode
3000    1.485915
3002    1.818182
3003    1.714286
3004    2.569892
3006    2.661538
Name: station_duration, dtype: float64

In [18]:
avg_park_dist = distances_df.groupby(by = "Postcode")["park_distance"].mean()
avg_park_dist.head()

Postcode
3000    1.000000
3002    1.000000
3003    1.000000
3004    1.129032
3006    1.066667
Name: park_distance, dtype: float64

In [20]:
avg_post_dur = distances_df.groupby(by = "Postcode")["postoffice_duration"].mean()
avg_post_dur.head()

Postcode
3000    1.380282
3002    1.909091
3003    2.000000
3004    1.666667
3006    2.302564
Name: postoffice_duration, dtype: float64

In [26]:
avg_num_schools = distances_df.groupby(by = "Postcode")["Nearby_Schools"].mean()
avg_num_schools.head()

Postcode
3000    613.021127
3002    628.068182
3003    591.206349
3004    564.387097
3006    630.579487
Name: Nearby_Schools, dtype: float64

In [28]:
# Creating dataframe of average amenities per postcode
amenities_df = pd.DataFrame()
amenities_df["Postcode"] = property_df["Postcode"].unique()
amenities_df["Average Duration to CBD"] = avg_cbd_dur.to_list()
amenities_df["Average Duration to Station"] = avg_station_dur.to_list()
amenities_df["Average Distance to Park"] = avg_park_dist.to_list()
amenities_df["Average Duration to Post Office"] = avg_post_dur.to_list()
amenities_df["Average # of Schools"] = avg_num_schools.to_list()
amenities_df.head()

,Postcode,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools
0,3000,130.582394,1.485915,1.000000,1.380282,613.021127
1,3002,302.665909,1.818182,1.000000,1.909091,628.068182
2,3003,228.284127,1.714286,1.000000,2.000000,591.206349
3,3004,431.089247,2.569892,1.129032,1.666667,564.387097
4,3006,311.044615,2.661538,1.066667,2.302564,630.579487


Calculating the average number of facilities

In [29]:
# Calculating the average number of facilities per postcode
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.601399
3002    1.909091
3003    1.796875
3004    1.763441
3006    1.785714
Name: Bed, dtype: float64

In [30]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.300699
3002    1.318182
3003    1.343750
3004    1.494624
3006    1.469388
Name: Bath, dtype: float64

In [31]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.356643
3002    1.000000
3003    0.609375
3004    1.096774
3006    0.647959
Name: Parking, dtype: float64

In [32]:
# Creating dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.601399,1.300699,0.356643
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.763441,1.494624,1.096774
4,3006,1.785714,1.469388,0.647959


In [33]:
# Joining facilities and amenities dataframes
amenities_df.set_index("Postcode", inplace = True)
facilities_df.set_index("Postcode", inplace = True)
postcode_property_df = amenities_df.join(facilities_df, on = "Postcode")
postcode_property_df.head()

,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,,
3000,130.582394,1.485915,1.000000,1.380282,613.021127,1.601399,1.300699,0.356643
3002,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000
3003,228.284127,1.714286,1.000000,2.000000,591.206349,1.796875,1.343750,0.609375
3004,431.089247,2.569892,1.129032,1.666667,564.387097,1.763441,1.494624,1.096774
3006,311.044615,2.661538,1.066667,2.302564,630.579487,1.785714,1.469388,0.647959


Obtaining census/population data

In [34]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [35]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [36]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing liveability
population_df = census_by_postcode_df[["postcode_2021", "tot_avg_hh_size_21"]]
population_df.head()

,postcode_2021,tot_avg_hh_size_21
0,3000,1.86
1,3002,1.87
2,3003,1.95
3,3004,1.84
4,3006,1.92


In [37]:
# Renaming postcode column of census dataframe and setting to index
population_df = population_df.rename({"postcode_2021": "Postcode"}, axis = 1)
population_df = population_df.set_index("Postcode")
population_df.head()

,tot_avg_hh_size_21
Postcode,
3000,1.86
3002,1.87
3003,1.95
3004,1.84
3006,1.92


In [38]:
# Joining census population dataframe and facilities and amenities dataframe
df = population_df.join(postcode_property_df, on = "Postcode")
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,,,
3000,1.86,130.582394,1.485915,1.000000,1.380282,613.021127,1.601399,1.300699,0.356643
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000
3003,1.95,228.284127,1.714286,1.000000,2.000000,591.206349,1.796875,1.343750,0.609375
3004,1.84,431.089247,2.569892,1.129032,1.666667,564.387097,1.763441,1.494624,1.096774
3006,1.92,311.044615,2.661538,1.066667,2.302564,630.579487,1.785714,1.469388,0.647959


Calculating liveability metric

In [39]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [44]:
TOTAL_PROPERTIES = len(property_df)
TOTAL_SCHOOLS = len(schools_df)

# Counting the number of properties per postcode
num_properties = property_df.groupby("Postcode").size()

In [45]:
# Calcuting the contribution of each postcode to the total number of rental properties as a proportion
df["Property Proportion"] = num_properties / TOTAL_PROPERTIES
df["Schools Proportion"] = df["Average # of Schools"] / TOTAL_SCHOOLS
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Schools Proportion
Postcode,,,,,,,,,,,,,,
3000,1.86,130.582394,1.485915,1.000000,1.380282,613.021127,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744,0.428986
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003328,1.020904,0.704910,0.534759,0.439516
3003,1.95,228.284127,1.714286,1.000000,2.000000,591.206349,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500,0.413720
3004,1.84,431.089247,2.569892,1.129032,1.666667,564.387097,1.763441,1.494624,1.096774,0.007034,0.958392,0.812295,0.596073,0.394952
3006,1.92,311.044615,2.661538,1.066667,2.302564,630.579487,1.785714,1.469388,0.647959,0.014825,0.930060,0.765306,0.337479,0.441273


In [46]:
# Calculating the average number of facilities per person in a household (for each postcode)
df["Beds per Person"] = df["Average # Beds"] / df["tot_avg_hh_size_21"]
df["Baths per Person"] = df["Average # Baths"] / df["tot_avg_hh_size_21"]
df["Parking per Person"] = df["Average # Parking"] / df["tot_avg_hh_size_21"]
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Schools Proportion
Postcode,,,,,,,,,,,,,,
3000,1.86,130.582394,1.485915,1.000000,1.380282,613.021127,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744,0.428986
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003328,1.020904,0.704910,0.534759,0.439516
3003,1.95,228.284127,1.714286,1.000000,2.000000,591.206349,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500,0.413720
3004,1.84,431.089247,2.569892,1.129032,1.666667,564.387097,1.763441,1.494624,1.096774,0.007034,0.958392,0.812295,0.596073,0.394952
3006,1.92,311.044615,2.661538,1.066667,2.302564,630.579487,1.785714,1.469388,0.647959,0.014825,0.930060,0.765306,0.337479,0.441273


In [47]:
# Calculating non-stadardised liveability metric
df["Liveability"] = ((1 / df["Average Duration to CBD"]) + df["Average Duration to Station"] + df["Average Distance to Park"] + 
                    df["Average Duration to Post Office"] + (df["Property Proportion"] * 100) + df["Beds per Person"] + df["Baths per Person"] + 
                    df["Parking per Person"] + df["Schools Proportion"])
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Schools Proportion,Liveability
Postcode,,,,,,,,,,,,,,,
3000,1.86,130.582394,1.485915,1.000000,1.380282,613.021127,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744,0.428986,7.136465
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003328,1.020904,0.704910,0.534759,0.439516,7.763470
3003,1.95,228.284127,1.714286,1.000000,2.000000,591.206349,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500,0.413720,7.539542
3004,1.84,431.089247,2.569892,1.129032,1.666667,564.387097,1.763441,1.494624,1.096774,0.007034,0.958392,0.812295,0.596073,0.394952,8.833050
3006,1.92,311.044615,2.661538,1.066667,2.302564,630.579487,1.785714,1.469388,0.647959,0.014825,0.930060,0.765306,0.337479,0.441273,9.990592


In [48]:
# Calculating standardised liveability metric
min_liveability = df.sort_values(by = "Liveability").head(1)["Liveability"].tolist()[0]
#print(min_liveability)
max_liveability = df.sort_values(by = "Liveability", ascending = False).head(1)["Liveability"].tolist()[0]
#print(max_liveability)

df["Standardised Liveability"] = (df["Liveability"] - min_liveability) / (max_liveability - min_liveability)

# Printing the top 10 most liveable suburbs
df.sort_values(by = "Standardised Liveability", ascending = False).head(10)

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Schools Proportion,Liveability,Standardised Liveability
Postcode,,,,,,,,,,,,,,,,
3213,2.49,3339.2,6.0,4.75,6.0,76.75,3.500000,1.750000,9.250000,0.000303,1.405622,0.702811,3.714859,0.053709,22.657556,1.000000
3688,2.40,12071.3,6.0,6.00,6.0,22.00,5.000000,3.000000,3.000000,0.000076,2.083333,1.250000,1.250000,0.015395,22.606375,0.997139
3808,2.82,3331.1,6.0,6.00,6.0,110.00,5.000000,4.000000,2.000000,0.000076,1.773050,1.418440,0.709220,0.076977,21.985550,0.962441
3670,2.40,7969.4,6.0,6.00,6.0,11.00,4.000000,1.000000,4.000000,0.000076,1.666667,0.416667,1.666667,0.007698,21.765387,0.950136
3757,2.80,3964.1,6.0,6.00,6.0,4.00,4.000000,2.000000,4.000000,0.000076,1.428571,0.714286,1.428571,0.002799,21.582044,0.939889
3799,2.30,5608.3,6.0,6.00,6.0,12.00,3.000000,3.000000,2.000000,0.000076,1.304348,1.304348,0.869565,0.008397,21.494400,0.934990
3249,2.46,7504.5,6.0,6.00,6.0,9.00,3.000000,1.000000,4.000000,0.000076,1.219512,0.406504,1.626016,0.006298,21.266028,0.922226
3659,2.47,4946.6,6.0,6.00,6.0,4.00,4.000000,2.000000,2.000000,0.000076,1.619433,0.809717,0.809717,0.002799,21.249431,0.921299
3461,2.35,4930.7,6.0,6.00,6.0,18.00,2.000000,1.500000,3.000000,0.000151,0.851064,0.638298,1.276596,0.012596,20.793884,0.895838


In [61]:
# Outputting liveability csv 
df = df.reset_index()
liveability_df = df[["Postcode", "Standardised Liveability"]]
liveability_df.to_csv("../data/curated/liveability.csv")

Graphing standardised liveability

In [28]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA2020_SHP/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)



gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

In [29]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

liveability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = liveability_df, # data source
            columns = ["Postcode", "Standardised Liveability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Liveability"
        )

c.add_to(m)
m.save(f"../plots/liveability_heatmap")

In [33]:
# Creating GeoPandas visualisation of top 10 most liveable postcodes
most_liveable_df = liveability_df.sort_values(by = "Standardised Liveability", ascending = False).head(10)

postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(most_liveable_df["Postcode"].to_list())]

In [34]:
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_liveable_heatmap")